#라이브러리 설치

In [ ]:
!git clone https://github.com/ultralytics/yolov5  
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99
!pip install -qr requirements.txt  
!pip install -q roboflow


In [52]:
import os
import yaml
import glob
import torch
from roboflow import Roboflow
from distutils.dir_util import copy_tree
from utils.downloads import attempt_download  # to download models/datasets
from IPython.display import Image, clear_output  # to display images
from IPython.core.magic import register_line_cell_magic

In [2]:
rf = Roboflow(model_format="yolov5", notebook="roboflow-yolov5")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.3 MB/s eta 0:00:00
Setup complete. Using torch 1.13.1+cu116 _CudaDeviceProperties(name='NVIDIA A100-SXM4-40GB', major=8, minor=0, total_memory=40536MB, multi_processor_count=108)


#데이터셋 다운로드

In [46]:
#데이터셋 다운로드
%cd /content/yolov5

rf = Roboflow(api_key="Ll2KOVai29VpLg3kFwfs")
project = rf.workspace("mohamed-traore-2ekkp").project("face-detection-mik1i")
dataset = project.version(20).download("yolov5")

project = rf.workspace("adrian-brandt").project("tattoos-pnxfx")
dataset = project.version(1).download("yolov5")

project = rf.workspace("unimonitor-03qbp").project("smoking-4")
dataset = project.version(1).download("yolov5")

project = rf.workspace("weapons-n32ov").project("weaponsdata")
dataset = project.version(4).download("yolov5")

#제일 마지막에 있는 데이터 셋을 중심으로 폴더를 합침

/content/yolov5
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Face-Detection-20 in yolov5pytorch:: 100%|██████████| 2750/2750 [00:00<00:00, 6518.18it/s]

loading Roboflow workspace...


loading Roboflow project...


Extracting Dataset Version Zip to tattoos-1 in yolov5pytorch:: 100%|██████████| 396/396 [00:00<00:00, 7565.43it/s]

...

loading Roboflow project...


Extracting Dataset Version Zip to smoking-4-1 in yolov5pytorch:: 100%|██████████| 4252/4252 [00:00<00:00, 7766.69it/s]

loading Roboflow workspace...


loading Roboflow project...


Extracting Dataset Version Zip to weaponsdata-4 in yolov5pytorch:: 100%|██████████| 18956/18956 [00:02<00:00, 7693.71it/s]


# 데이터셋 통합 작업(클래스 변경)

아래 작업과정 설명
A 데이터셋에 1,2,3 클래스가 있고
B 데이터셋에 1,2 클래스가 있으면

B 데이터셋을 A 데이터 셋에 통합하는 과정

1을 통해 클래스의 개수 확인

2를 통해 클래스 전환 B의 1클래스를 4로 2클래스를 5로 전환

3를 통해 중복되는 데이터 파일의 이름 존재 여부를 판단

4를 통해 통합

결과는 A 데이터셋에 1,2,3,4,5 클래스가 생김

In [49]:
#1
#옮길 데이터셋과 옮겨질 위치의 데이터셋 이름
#ex) ./from_data 폴더 내의 모든 파일들을 ./to_data 폴더로 모두 옮기면서 덮어씀(이때 데이터셋 파일(이미지,라벨)은 겹치면 안됨)
from_data = 'Face-Detection-20'
to_data = 'weaponsdata-4'

#해당 데이터셋에 특정 클래스가 몇개 있는지 확인

from_target_class = '0'
to_target_class = '5'

train = glob.glob(f'/content/yolov5/{from_data}/train/labels/*.txt')
valid = glob.glob(f'/content/yolov5/{from_data}/valid/labels/*.txt')
test = glob.glob(f'/content/yolov5/{from_data}/test/labels/*.txt')
count=0
for file_path in train+valid+test:
    with open(file_path,'r') as f:
        lines = f.readlines()
    for i in range(len(lines)):
        if lines[i].find(from_target_class, 0,1) == 0:
          count+= 1

print('From:',count)

train = glob.glob(f'/content/yolov5/{to_data}/train/labels/*.txt')
valid = glob.glob(f'/content/yolov5/{to_data}/valid/labels/*.txt')
test = glob.glob(f'/content/yolov5/{to_data}/test/labels/*.txt')
count=0
for file_path in train+valid+test:
    with open(file_path,'r') as f:
        lines = f.readlines()
    for i in range(len(lines)):
        if lines[i].find(to_target_class, 0,1) == 0:
          count+= 1
print('To:',count)

From: 2396
To: 251


In [ ]:
#2
#from data에 있는 from_target_class 번호를 to_target_class로 변경

#만약 변경해야할 클래스수가 많다면 위에서부터 진행

train = glob.glob(f'/content/yolov5/{from_data}/train/labels/*.txt')
valid = glob.glob(f'/content/yolov5/{from_data}/valid/labels/*.txt')
test = glob.glob(f'/content/yolov5/{from_data}/test/labels/*.txt')
for file_path in train+valid+test:
    with open(file_path,'r') as f:
        lines = f.readlines()
    s = ''
    for i in range(len(lines)):
        s += lines[i].replace(from_target_class,to_target_class,1) +'\n' 
    with open(file_path,'w') as f:
        f.write(s)


In [39]:
# 3
# 중복파일이 있는지(다른 데이터셋에 중복된 이름이 존재하면 데이터가 대체되기 때문이다. 
# 큰 문제는 없지만 모든 데이터를 사용하는게 좋기때문에 확인함

train = glob.glob(f'/content/yolov5/{from_data}/train/labels/*.txt')
valid = glob.glob(f'/content/yolov5/{from_data}/valid/labels/*.txt')
test = glob.glob(f'/content/yolov5/{from_data}/test/labels/*.txt')

for file_path in train+valid+test:
    if os.path.exists(file_path.replace(from_data,to_data)):
      print(f'{file_path}\n{file_path.replace(from_data,to_data)}\n파일이 존재합니다.\n')

In [ ]:
# 4
# 문제가 없다면 셀을 실행해 통합 진행
copy_tree(f'/content/yolov5/{from_data}/',f'/content/yolov5/{to_data}/')

In [56]:
#데이터를 합치고 난 후 class 수(nc) 각 class별 이름(names) 설정 

%cat {dataset.location}/data.yaml

names: ['knife', 'long_gun', 'pistol','face','smoking','tatoo']
nc: 6
train: ./weaponsdata-4/train/images
val: ./weaponsdata-4/valid/images
test: ./weaponsdata-4/test/images


In [57]:
#Yaml을 참고해서  클래스 수 설정 그냥 필요없이 num_classes 값 정의 해줘도 됨
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = yaml.safe_load(stream)['nc']

In [58]:
#모델 설정
%cat /content/yolov5/models/yolov5m.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.67  # model depth multiple
width_multiple: 0.75  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]]

In [59]:
@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [60]:
%%writetemplate /content/yolov5/models/custom_yolov5m.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [61]:
# 데이터 셋 위치인데 적절히 수정해도 상관없음
dataset.location

'/content/yolov5/weaponsdata-4'

In [64]:
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 32 --epochs 500 --data {dataset.location}/data.yaml --cfg ./models/custom_yolov5m.yaml --weights yolov5m.pt --name mosaic  --cache

/content/yolov5
train: weights=yolov5m.pt, cfg=./models/custom_yolov5m.yaml, data=/content/yolov5/weaponsdata-4/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=500, batch_size=32, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=mosaic, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 436 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (NVIDIA A100-SXM4-40GB, 40536MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum